In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Read values
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

# Initialize OpenAI-compatible client
client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

print("Config loaded from .env")

Config loaded from .env


In [2]:
import os
print("Working directory:", os.getcwd())
print("Files here:", os.listdir())

Working directory: /app
Files here: ['.dockerignore', '.env', '.git', '.gitignore', '.ipynb_checkpoints', 'config.yaml', 'docker-compose.yaml', 'Dockerfile', 'python', 'README.md', 'requirements.txt']


In [3]:
import importlib.util

spec = importlib.util.spec_from_file_location("schema_models", "/app/python/schema_models.py")
schema_models = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schema_models)

SchemaPrompt = schema_models.SchemaPrompt
SchemaObject = schema_models.SchemaObject
ColumnSchema = schema_models.ColumnSchema

In [4]:
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
from pydantic import ValidationError
import os

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)

def test_llm(client):
    test = client.chat.completions.create(
        model="deepseek-chat",
        messages=[{"role": "user", "content": "Say hello"}]
    )
    print("✅ Test LLM response:", test.choices[0].message.content)


class SchemaAgent:
    def __init__(self, llm_client: OpenAI):
        self.llm = llm_client

    def generate_from_prompt(self, schema_prompt: SchemaPrompt) -> SchemaObject:
        assert schema_prompt.prompt, "Prompt is required"
        system_msg = (
            "You are a strict schema generator. Return ONLY a JSON object like:\n"
            "{\n"
            "  \"columns\": [\n"
            "    {\"name\": \"age\", \"type\": \"int\", \"min\": 0, \"max\": 120},\n"
            "    {\"name\": \"gender\", \"type\": \"categorical\", \"values\": [\"M\", \"F\"]},\n"
            "    {\"name\": \"admission_date\", \"type\": \"datetime\", \"format\": \"%Y-%m-%d\"}\n"
            "  ]\n"
            "}"
        )
        user_msg = f"Use-case: {schema_prompt.use_case}\nPrompt: {schema_prompt.prompt}"

        response = self.llm.chat.completions.create(
            model="deepseek-chat",
            messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
            ]
        )

        text = response.choices[0].message.content
        if not text or not text.strip():
            raise ValueError("❌ Empty response from LLM. Check API key, base URL, or network.")
        print("✅ LLM Output:\n", text)
        if not text or not text.strip():
            raise ValueError("LLM response is empty or invalid. Check API status or quota.")

        try:
            # Try to extract valid JSON from potentially messy output
            json_start = text.find('{')
            json_end = text.rfind('}') + 1
            parsed = json.loads(text[json_start:json_end])
            return SchemaObject(use_case=schema_prompt.use_case, **parsed)
        except (json.JSONDecodeError, ValidationError) as e:
            print(f"❌ LLM output invalid: {e}")
            raise ValueError(f"LLM returned malformed or invalid schema.\nRaw output:\n{text}")

    def generate_from_csv(self, schema_prompt: SchemaPrompt) -> SchemaObject:
        assert schema_prompt.csv_path, "CSV path is required"
        df = pd.read_csv(schema_prompt.csv_path)
        cols = []

        for col in df.columns:
            dtype = df[col].dtype
            col_type = "string"
            if pd.api.types.is_integer_dtype(dtype):
                col_type = "int"
            elif pd.api.types.is_float_dtype(dtype):
                col_type = "float"
            elif pd.api.types.is_datetime64_any_dtype(dtype):
                col_type = "datetime"
            elif pd.api.types.is_categorical_dtype(dtype) or df[col].nunique() < 10:
                col_type = "categorical"

            col_schema = ColumnSchema(
                name=col,
                type=col_type,
                min=float(df[col].min()) if col_type in ["int", "float"] else None,
                max=float(df[col].max()) if col_type in ["int", "float"] else None,
                values=list(map(str, df[col].dropna().unique())) if col_type == "categorical" else None,
                format="%Y-%m-%d" if col_type == "datetime" else None
            )
            cols.append(col_schema)

        return SchemaObject(use_case=schema_prompt.use_case, columns=cols)


In [5]:
schema_prompt = SchemaPrompt(
    use_case="Employee record generation",
    prompt="Generate a schema for employee records including age (18-65), gender (M/F), role, and join date"
)

schema_agent = SchemaAgent(llm_client=client)  # `client` = your DeepSeek OpenAI-compatible instance

schema = schema_agent.generate_from_prompt(schema_prompt)

print(schema)

✅ LLM Output:
 ```json
{
  "columns": [
    {"name": "age", "type": "int", "min": 18, "max": 65},
    {"name": "gender", "type": "categorical", "values": ["M", "F"]},
    {"name": "role", "type": "string"},
    {"name": "join_date", "type": "datetime", "format": "%Y-%m-%d"}
  ]
}
```
use_case='Employee record generation' columns=[ColumnSchema(name='age', type='int', min=18.0, max=65.0, format=None, values=None), ColumnSchema(name='gender', type='categorical', min=None, max=None, format=None, values=['M', 'F']), ColumnSchema(name='role', type='string', min=None, max=None, format=None, values=None), ColumnSchema(name='join_date', type='datetime', min=None, max=None, format='%Y-%m-%d', values=None)]


In [7]:
!pip install ctgan


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 12.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 13.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 12.5 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 15.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 13.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 11.0 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
import pandas as pd
from ctgan import CTGAN
from typing import List
from faker import Faker
import random

class CTGANGeneratorAgent:
    def __init__(self):
        self.faker = Faker()
        self.model = CTGAN(epochs=100)
        self.generated_categories = {}

    def get_dynamic_category(self, col_name: str, generator_fn, max_unique=200):
        if col_name not in self.generated_categories:
            self.generated_categories[col_name] = list({generator_fn() for _ in range(max_unique)})
        return random.choice(self.generated_categories[col_name])
    
    
    def generate_fake_data_from_schema(self, schema: SchemaObject, n=100) -> pd.DataFrame:
        rows = []
        for _ in range(n):
            row = {}
            for col in schema.columns:
                if col.type == "int":
                    row[col.name] = random.randint(int(col.min or 0), int(col.max or 100))
                elif col.type == "float":
                    row[col.name] = round(random.uniform(col.min or 0.0, col.max or 100.0), 2)
                elif col.type == "categorical":
                    row[col.name] = random.choice(col.values or ["Unknown"])
                elif col.type == "datetime":
                    fmt = col.format or "%Y-%m-%d"
                    row[col.name] = self.faker.date_between(start_date='-5y', end_date='today').strftime(fmt)
                elif col.type == "string":
                    if "name" in col.name.lower():
                        row[col.name] = self.get_dynamic_category(col.name, self.faker.name)
                    elif "city" in col.name.lower():
                        row[col.name] = self.get_dynamic_category(col.name, self.faker.city)
                    elif "email" in col.name.lower():
                        row[col.name] = self.get_dynamic_category(col.name, lambda: f"{self.faker.first_name().lower()}{random.randint(1,99)}@gmail.com")
                    else:
                        row[col.name] = self.get_dynamic_category(col.name, self.faker.word)
                else:
                    row[col.name] = None
            rows.append(row)
        return pd.DataFrame(rows)

    def fit_ctgan_on_fake_data(self, fake_df: pd.DataFrame, schema: SchemaObject):
        cat_cols = [col.name for col in schema.columns if col.type == "categorical"]
        self.model.fit(fake_df, discrete_columns=cat_cols)

    def sample(self, n=100) -> pd.DataFrame:
        return self.model.sample(n)

    def generate_from_schema(self, schema: SchemaObject, n=100) -> pd.DataFrame:
        fake_df = self.generate_fake_data_from_schema(schema, n=100)
    
        # Keep only valid CTGAN columns
        valid_types = ["string","int", "float", "categorical"]
        safe_cols = [col.name for col in schema.columns if col.type in valid_types]
        fake_df = fake_df[safe_cols]

        self.fit_ctgan_on_fake_data(fake_df, schema)
        return self.sample(n)

In [18]:


schema_prompt = SchemaPrompt(
    use_case="Retail customer data",
    prompt="Create schema with name,place ,age (18-60), gender (M/F), city, salary (30k-150k), join_date"
)

schema = SchemaAgent(llm_client=client).generate_from_prompt(schema_prompt)

gen = CTGANGeneratorAgent()
df = gen.generate_from_schema(schema, n=5)

print(df)


✅ LLM Output:
 ```json
{
  "columns": [
    {"name": "name", "type": "string"},
    {"name": "place", "type": "string"},
    {"name": "age", "type": "int", "min": 18, "max": 60},
    {"name": "gender", "type": "categorical", "values": ["M", "F"]},
    {"name": "city", "type": "string"},
    {"name": "salary", "type": "int", "min": 30000, "max": 150000},
    {"name": "join_date", "type": "datetime", "format": "%Y-%m-%d"}
  ]
}
```


TypeError: Could not convert string 'Kim MorrisAdam RoyJenna MooreOlivia JonesAmy DavisJoe JeffersonJennifer AdamsBrittany MitchellBethany NobleMiss Emily HowardChristopher AndersonTerry BarkerKerry DiazElizabeth PetersJennifer WarrenGeorge JohnsonMichelle AdamsAlexandra BoydTerri NelsonAlexander WatsonDenise NelsonCrystal JacksonGary TranChristopher AndersonMegan JenkinsDerek MyersKathryn VillaAmanda ThompsonDaniel BerryApril EvansBrittany KnightErin BeckerAlexander WatsonAmanda ThompsonMelissa WilsonBethany NobleAmber GardnerDavid GallowayAndrew BoydDominic FigueroaErin BeckerLori Levine MDRobert LeeCameron WaltersMaria JohnsonLori Levine MDTimothy BarnesSamantha KimLauren WalshChad LeeKristin JacksonMadison MaddenLaura HallDebra GrayBrandi AllenAmber GardnerTerry BarkerTerri NelsonAmanda ThompsonLuis SmithTimothy BarnesMark MeadowsMelissa WilsonJoe JeffersonElizabeth RichardsonMr. Daniel HarrisAndrew PopeBrittany MitchellSusan ShawRodney FigueroaNicole Collins DVMBrittany AlvarezTon